# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [1]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicsp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicsp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [22]:
# define function that removes URL from single tweet
def remove_urls_mentions_RT(text):
    
    text1 = re.sub(r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+', "link", text)
    text2 = re.sub(r'@(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+', "", text1)
    text3 = re.sub(r'^RT[\s]+', '', text2)
    
    return text3

**Original object:**

In [49]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [44]:
remove_urls_mentions_RT(tweet)

'Researchers say we have three years to act on climate change before it’s too late  …'

**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [4]:
def to_lower(text):
    text = text.lower()
    return text

**Original object:**

In [8]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [9]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [5]:
def get_roots(text):
    
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)
    
    return text

In [53]:
def get_roots(text):
    
    stemmer = SnowballStemmer(language = 'english')
    text = stemmer.stem(text)
    
    return text

**Original object:**

In [51]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [54]:
get_roots(tweet)

"rt @rawstory: researchers say we have three years to act on climate change before it's too late https://t.co/wdt0kdur2f https://t.co/z0anpt…"

**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [6]:
#remove punctuation and numbers

import string

def remove_punctuation_numbers(text):
    punc_numbers = string.punctuation + '0123456789'
    
    return ''.join([l for l in text if l not in punc_numbers])

In [13]:
remove_punctuation_numbers(tweet)

'RT RawStory Researchers say we have three years to act on climate change before it’s too late httpstcoWdTKdUrf httpstcoZANPT…'

### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [7]:
def get_tokens(text):
    tokeniser = TreebankWordTokenizer()
    text = tokeniser.tokenize(text) 
    return text

In [78]:
def get_tokens(text):
    tokeniser = TweetTokenizer()
    text = tokeniser.tokenize(text) 
    return text

**Original object:**

In [79]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [80]:
get_tokens(tweet)

['RT',
 '@RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it',
 '’',
 's',
 'too',
 'late',
 'https://t.co/WdT0KdUr2f',
 'https://t.co/Z0ANPT…']

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [38]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [49]:
# clean tweet from start to finish
tweet1 = remove_urls_mentions_RT(tweet)
tweet2 = to_lower(tweet1)
tweet3 = remove_punctuation_numbers(tweet2)
tweet4 = get_roots(tweet3)
tweet_final = get_tokens(tweet4)

print(tweet_final)

['researchers', 'say', 'we', 'have', 'three', 'years', 'to', 'act', 'on', 'climate', 'change', 'before', 'it’s', 'too', 'late', '…']


**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [17]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [82]:
# applying cleaning functions to entire column of dataframe (Train - NO VECTORISING)

train['message_clean'] = train['message'].apply(remove_urls_mentions_RT)

train['message_clean'] = train['message_clean'].apply(to_lower)

train['message_clean'] = train['message_clean'].apply(remove_punctuation_numbers)

train['message_clean'] = train['message_clean'].apply(get_roots)

train['message_clean'] = train['message_clean'].apply(get_tokens)

In [83]:
# applying cleaning functions to entire column of dataframe (Test - NO VECTORISING)

test['message_clean'] = test['message'].apply(remove_urls_mentions_RT)

test['message_clean'] = test['message_clean'].apply(to_lower)

test['message_clean'] = test['message_clean'].apply(remove_punctuation_numbers)

test['message_clean'] = test['message_clean'].apply(get_roots)

test['message_clean'] = test['message_clean'].apply(get_tokens)

In [11]:
train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [84]:
# Separate minority and majority classes
One = train[train['sentiment']==1]
Two = train[train['sentiment']==2]
Zero = train[train['sentiment']==0]
Minus_one = train[train['sentiment']==-1]

In [85]:
#downsampling
from sklearn.utils import resample

One_downsampled = resample(One,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=4000, # match number in minority class
                          random_state=27) # reproducible results

Two_upsampled = resample(Two,
                          replace=True, # sample without replacement (no need to duplicate observations)
                          n_samples=4000, # match number in minority class
                          random_state=27) # reproducible results

Zero_upsampled = resample(Zero,
                          replace=True, # sample without replacement (no need to duplicate observations)
                          n_samples=4000, # match number in minority class
                          random_state=27) # reproducible results

Minus_one_upsampled = resample(Minus_one,
                          replace=True, # sample without replacement (no need to duplicate observations)
                          n_samples=4000, # match number in minority class
                          random_state=27) # reproducible results

# Combine downsampled classes with upsampled class
train_downsampled = pd.concat([One_downsampled, Two_upsampled, Zero_upsampled, Minus_one_upsampled])

In [62]:
train_downsampled['sentiment'].value_counts()

 1    4000
 2    4000
 0    4000
-1    4000
Name: sentiment, dtype: int64

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [86]:
 tt = TfidfVectorizer(preprocessor=list, tokenizer=list, ngram_range=(1,2), min_df=2, strip_accents = 'ascii', stop_words = 'english', smooth_idf=False)

In [87]:
# vectorise your cleaned data

tt.fit(train_downsampled['message_clean'])

train_vec = tt.transform(train_downsampled['message_clean'])

# YOUR CODE HERE

C:\Users\nicsp\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\nicsp\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


In [88]:
test_vec = tt.transform(test['message_clean'])

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [89]:
# YOUR CODE HERE 

# labels
y = train_downsampled['sentiment']

# features
X = train_vec.toarray()

#split into training and validation data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [90]:
#model logistic regression

lr = LogisticRegression(solver = 'liblinear', random_state = 50)
lr.fit(X_train, y_train)

LogisticRegression(random_state=50, solver='liblinear')

In [91]:
#predictions

pred_lr = lr.predict(X_test)

f1_score_train = f1_score(y_train, lr.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_lr, average='macro')

print(f1_score_train, f1_score_test)

0.9172130527187994 0.8417231212431525


In [33]:
#model logistic regression more hyperparameters

lr2 = LogisticRegression(solver = 'liblinear', C=2, random_state = 50)
lr2.fit(X_train, y_train)

pred_lr2 = lr2.predict(X_test)

f1_score_train = f1_score(y_train, lr2.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_lr2, average='macro')

print(f1_score_train, f1_score_test)

0.9555612756209775 0.8596656219458216


In [60]:
#model tree

tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)

pred_tree = tree.predict(X_test)

f1_score_train = f1_score(y_train, tree.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_tree, average='macro')

print(f1_score_train, f1_score_test)

0.43635834668231577 0.4410800483705124


In [74]:
#model random forest

forest = RandomForestClassifier(max_depth=8, random_state=42, n_estimators=100,  max_features=100)
forest.fit(X_train, y_train)

pred_forest = forest.predict(X_test)

f1_score_train = f1_score(y_train, forest.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_forest, average='macro')

print(f1_score_train, f1_score_test)

0.6205368767482227 0.5849307468000412


In [75]:
#modal SVM linear

Linear_SVM = SVC(kernel="linear", C=0.025)
Linear_SVM.fit(X_train, y_train)

pred_Linear_SVM = Linear_SVM.predict(X_test)

f1_score_train = f1_score(y_train, Linear_SVM.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_Linear_SVM, average='macro')

print(f1_score_train, f1_score_test)

0.3329826864446353 0.3164548262783372


In [20]:
RBF_SVM = SVC(gamma=2, C=1)
RBF_SVM.fit(X_train, y_train)

pred_RBF_SVM = RBF_SVM.predict(X_test)

f1_score_train = f1_score(y_train, RBF_SVM.predict(X_train), average='macro')
f1_score_test = f1_score(y_test, pred_RBF_SVM, average='macro')

print(f1_score_train, f1_score_test)

0.9950803805727684 0.8769472788802533


## Prepare submission

In [44]:
test_vec_array = test_vec.toarray()

In [92]:
#predict on the cleaned test data

pred_test = lr.predict(test_vec)
predictions = pd.DataFrame(pred_test)

In [93]:
# include tweetids
ids_submit = test[['tweetid']]

# create dataframe and name columns as needed
submit_all = pd.concat([ids_submit.reset_index(drop=True), predictions], axis=1).rename(columns = {0:'sentiment'})
submit_all.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0


In [94]:
#write submissions to CSV
submit_all.to_csv(r'prediction_submission12.csv', index = False)

## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

In [ ]:
# Extras
# YOUR CODE HERE 




## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

In [ ]:
# EDA
# YOUR CODE HERE 





Be sure to check whether each additional technique you apply actually improves your model performance.

## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [ ]:
# write csv for best predictions


